In [19]:
import geopandas as geopd
import pandas as pd
import json
import folium

In [2]:
LODES = pd.read_csv("/home/data/census/nyc/LODES/ny_od_main_JT01_2019.csv")
ntas = geopd.read_file("https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Neighborhood_Tabulation_Areas_2020/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson")
crosswalk = pd.read_excel("https://www.nyc.gov/assets/planning/download/office/planning-level/nyc-population/census2020/nyc2020census_tract_nta_cdta_relationships.xlsx?r=092221")

In [3]:
LODES["w_geocode"] = LODES["w_geocode"].astype(str)
LODES["tract_GEOID_work"] = LODES.w_geocode.str.slice(0,11)

LODES["h_geocode"] = LODES["h_geocode"].astype(str)
LODES["tract_GEOID_res"] = LODES.h_geocode.str.slice(0,11)

In [4]:
# Summarise at tract level for merge
tract_jobs = LODES.groupby(["tract_GEOID_work", "tract_GEOID_res"]).agg(total_jobs = ("S000", "sum")).reset_index()

In [5]:
# Adapt crosswalk for easy merge

crosswalk["GEOID"] = crosswalk["GEOID"].astype(str)
crosswalk["tract_GEOID_work"] = crosswalk["GEOID"]
crosswalk["tract_GEOID_res"] = crosswalk["GEOID"]
crosswalk["NTACode_work"] = crosswalk["NTACode"]
crosswalk["NTACode_res"] = crosswalk["NTACode"]

In [6]:
nta_jobs = (tract_jobs
            .merge(crosswalk[["tract_GEOID_res", "NTACode_res"]])
            .merge(crosswalk[["tract_GEOID_work", "NTACode_work"]])
            .groupby(["NTACode_res", "NTACode_work"])
            .agg(total_jobs = ("total_jobs", "sum"))
           .reset_index())

In [50]:
def make_job_map(nta_code, nta_jobs, nta_shapes):
    nta = nta_jobs[nta_jobs["NTACode_res"] == nta_code]
    
    map_dat = nta_shapes.merge(nta, right_on = "NTACode_work", left_on = "NTA2020")

    base_map = folium.Map(location=[40.730610, -73.935242], zoom_start=10)

    folium.Choropleth(
        geo_data=map_dat,
        data=map_dat,
        columns=["NTA2020", "total_jobs"],
        key_on="feature.properties.NTA2020",
        fill_color="BuPu",
        fill_opacity=0.7,
        line_opacity=1,
        legend_name="Total Jobs",
        reset=True,
    ).add_to(base_map)
    
    return base_map

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape__Area,Shape__Length,geometry
0,1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),3.532193e+07,28914.315199,"POLYGON ((-73.93214 40.72817, -73.93253 40.727..."
1,2,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),2.886227e+07,28155.614512,"POLYGON ((-73.95814 40.72441, -73.95772 40.724..."
2,3,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),1.520934e+07,18252.346181,"POLYGON ((-73.95024 40.70548, -73.94984 40.705..."
3,4,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),5.226618e+07,43184.828736,"POLYGON ((-73.92406 40.71412, -73.92404 40.714..."
4,5,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,9.985438e+06,14337.853843,"POLYGON ((-73.99237 40.68970, -73.99436 40.690..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,258,5,Staten Island,085,SI0391,Freshkills Park (South),FrshklPK_S,9,SI03,SI03 South Shore (CD 3 Approximation),4.775867e+07,33945.050028,"POLYGON ((-74.20059 40.57952, -74.19888 40.579..."
258,259,5,Staten Island,085,SI9561,Fort Wadsworth,FtWdswrth,6,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,9.883809e+06,14852.484590,"POLYGON ((-74.05975 40.59386, -74.06014 40.594..."
259,260,5,Staten Island,085,SI9591,Hoffman & Swinburne Islands,HffmnIsl,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,6.357020e+05,4743.128127,"MULTIPOLYGON (((-74.05314 40.57771, -74.05406 ..."
260,261,5,Staten Island,085,SI9592,Miller Field,MllrFld,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,1.086771e+07,19196.371774,"POLYGON ((-74.08469 40.57149, -74.08595 40.570..."


In [68]:
make_job_map("QN0704", nta_jobs, ntas)

In [58]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ntas[["NTA2020", "NTAName"]])

    NTA2020                                            NTAName
0    BK0101                                         Greenpoint
1    BK0102                                       Williamsburg
2    BK0103                                 South Williamsburg
3    BK0104                                  East Williamsburg
4    BK0201                                   Brooklyn Heights
5    BK0202                Downtown Brooklyn-DUMBO-Boerum Hill
6    BK0203                                        Fort Greene
7    BK0204                                       Clinton Hill
8    BK0261                                 Brooklyn Navy Yard
9    BK0301                          Bedford-Stuyvesant (West)
10   BK0302                          Bedford-Stuyvesant (East)
11   BK0401                                    Bushwick (West)
12   BK0402                                    Bushwick (East)
13   BK0471                            The Evergreens Cemetery
14   BK0501                                      Cypres

In [57]:
ntas

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape__Area,Shape__Length,geometry
0,1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),3.532193e+07,28914.315199,"POLYGON ((-73.93214 40.72817, -73.93253 40.727..."
1,2,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),2.886227e+07,28155.614512,"POLYGON ((-73.95814 40.72441, -73.95772 40.724..."
2,3,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),1.520934e+07,18252.346181,"POLYGON ((-73.95024 40.70548, -73.94984 40.705..."
3,4,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),5.226618e+07,43184.828736,"POLYGON ((-73.92406 40.71412, -73.92404 40.714..."
4,5,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,9.985438e+06,14337.853843,"POLYGON ((-73.99237 40.68970, -73.99436 40.690..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,258,5,Staten Island,085,SI0391,Freshkills Park (South),FrshklPK_S,9,SI03,SI03 South Shore (CD 3 Approximation),4.775867e+07,33945.050028,"POLYGON ((-74.20059 40.57952, -74.19888 40.579..."
258,259,5,Staten Island,085,SI9561,Fort Wadsworth,FtWdswrth,6,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,9.883809e+06,14852.484590,"POLYGON ((-74.05975 40.59386, -74.06014 40.594..."
259,260,5,Staten Island,085,SI9591,Hoffman & Swinburne Islands,HffmnIsl,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,6.357020e+05,4743.128127,"MULTIPOLYGON (((-74.05314 40.57771, -74.05406 ..."
260,261,5,Staten Island,085,SI9592,Miller Field,MllrFld,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,1.086771e+07,19196.371774,"POLYGON ((-74.08469 40.57149, -74.08595 40.570..."
